How should we handle a stochastic model? In computing the sensitivity measures, is it sufficient to run the model *once* for each sample parameter set, should we use an average over many runs for each sample parameter set, or maybe even just a few runs for each parameter set?
* Try this same sort of analysis using persistence time (and maybe mean number infected over 10 years) as the model outcome.

# Persistence time of a general epidemic

The stochastic general epidemic model with counts $(S, I, R)$ of the number of people susceptible, infectious, and recovered, has events:

* Birth: $(S, I, R) \to (S + 1, I, R)$ with hazard $\mu N$;
* Death of a susceptible: $(S, I, R) \to (S - 1, I, R)$ with hazard $\mu S$;
* Death of an infectious: $(S, I, R) \to (S, I - 1, R)$ with hazard $\mu I$;
* Death of a recovered: $(S, I, R) \to (S, I, R - 1)$ with hazard $\mu R$;
* Infection: $(S, I, R) \to (S - 1, I + 1, R)$ with hazard $\frac{\beta I}{N} S$; and
* Recovery: $(S, I, R) \to (S, I - 1, R + 1)$ with hazard $\gamma I$;

where $N = S + I + R$.

The basic reproduction number is
$$R_0 = \frac{\beta}{\gamma + \mu}.$$

We will use the endemic equilibrium of the deterministic model
$$\begin{aligned}
\frac{\mathrm{d} S}{\mathrm{d} t} &= \mu N - \frac{\beta I}{N} S - \mu S,\\
\frac{\mathrm{d} I}{\mathrm{d} t} &= \frac{\beta I}{N} S - \gamma I - \mu I,\\
\frac{\mathrm{d} R}{\mathrm{d} t} &= \gamma I - \mu R,
\end{aligned}$$
as the expected value of a multinomial distribution to determine the initial condition for the stochastic model:
$$\left(S(0), I(0), R(0)\right)
= \mathrm{Multinomial}(N_0, p),$$
where
$$p = \left(\frac{1}{R_0},
            \frac{\mu}{\beta}\left(R_0 - 1\right),
            \frac{\gamma}{\beta}\left(R_0 - 1\right)\right),$$
and $N_0$ is the initial population size.
We will resample from this multinomial we get an initial condition with $I(0) > 0$.

We will simulate the stochastic model using the Gillespie algorithm.

For each simulation, we will find the extinction time, the first time with $I(t) = 0$.

In [1]:
import joblib
import numpy
from scipy import stats

import sensitivity_analysis

In [30]:
def get_R_0(beta, gamma, mu, N):
    return beta / (gamma + mu)


def initial_state(beta, gamma, mu, N_0, rng):
    '''Generate a random initial `state`.'''
    R_0 = get_R_0(beta, gamma, mu, N_0)
    p = numpy.array((1 / R_0,                    # S
                     mu / beta * (R_0 - 1),      # I
                     gamma / beta * (R_0 - 1)))  # R
    # If R_0 > 1, resample to get I > 0.
    while True:
        state = rng.multinomial(N_0, p)
        (S, I, R) = state
        if (R_0 <= 1) or (I > 0):
            break
    return state


# The transitions that can occur and how they change `state`.
transitions = numpy.array((
    (+1, 0, 0),   # Birth
    (-1, 0, 0),   # Death of S
    (0, -1, 0),   # Death of I
    (0, 0, -1),   # Death of R
    (-1, +1, 0),  # Infection
    (0, -1, +1),  # Recovery
))


def update_hazards(hazards, t, state, beta, gamma, mu):
    '''Update `hazards` for the current state and time.'''
    (S, I, R) = state
    N = state.sum()
    # `hazards` must be in the same order as `transitions`.
    hazards[:] = (
        mu * N,            # Birth
        mu * S,            # Death of S
        mu * I,            # Death of I
        mu * R,            # Death of R
        beta * I / N * S,  # Infection
        gamma * I,         # Recovery
    )


def stop(t, state):
    '''The stopping condition for the simulation.'''
    (S, I, R) = state
    return (I == 0)


def get_persistence_time(beta, gamma, mu, N_0=1000, seed=None):
    '''Simulate the persistence time for a stochastic general SIR model.'''
    rng = numpy.random.default_rng(seed)
    t = 0
    state = initial_state(beta, gamma, mu, N_0, rng)
    # Build empty vectors that will get updated in each step.
    n_transitions = len(transitions)
    hazards = numpy.empty(n_transitions, dtype=float)
    hazards_scaled = numpy.empty(n_transitions, dtype=float)
    while not numpy.isposinf(t) and not stop(t, state):
        update_hazards(hazards, t, state, beta, gamma, mu)
        # Find the time to the next event.
        hazard_total = hazards.sum()
        if hazard_total > 0:
            t += rng.exponential(1 / hazard_total)
            # Find which of the events occurred.
            # Scale the hazards so that they sum to 1.
            hazards_scaled[:] = hazards / hazard_total
            which = rng.choice(n_transitions, p=hazards_scaled)
            state += transitions[which]
        else:  # hazard_total == 0
            # Check that we don't have hazard_total < 0.
            assert numpy.isclose(hazard_total, 0)
            t = numpy.PINF
    return t

In [31]:
get_persistence_time(0.012, 0.01, 0.001, seed=1)

2171.1930732740902

Let's take the parameters to be the random variables
$$\begin{aligned}
\beta &\sim \Gamma(0.03, 4),\\
\gamma &\sim \Gamma(0.01, 4),\\
\mu &\sim \Gamma(0.001, 4),
\end{aligned}$$
where $\Gamma(a, k)$ is the gamma random variable with mean $a$ and shape $k$.

In [32]:
def gamma_from_mean_and_shape(mean, shape):
    '''Generate a gamma random variable with the given mean and shape.'''
    scale = mean / shape
    return stats.gamma(shape, scale=scale)


transmission_rate = gamma_from_mean_and_shape(0.03, 4)
recovery_rate = gamma_from_mean_and_shape(0.01, 4)
death_rate = gamma_from_mean_and_shape(0.001, 4)
parameters = dict(beta=transmission_rate,
                  gamma=recovery_rate,
                  mu=death_rate)

In [33]:
def get_persistence_times(beta, gamma, mu, seed=None, n_jobs=-1):
    '''Get the simulated persistence time for sequences of parameters.'''
    if isinstance(seed, numpy.random.SeedSequence):
        seed_seq = seed
    else:
        seed_seq = numpy.random.SeedSequence(entropy=seed)
    n_samples = len(beta)
    seeds = seed_seq.spawn(n_samples)
    with joblib.Parallel(n_jobs=n_jobs) as parallel:
        return parallel(
            joblib.delayed(get_persistence_time)(*args, seed=seed)
            for (*args, seed) in zip(beta, gamma, mu, seeds))

In [34]:
n_samples = 2
seed_seq = numpy.random.SeedSequence(1)
seeds = seed_seq.spawn(2)
samples = sensitivity_analysis.samples_Latin_hypercube(parameters, n_samples, seed=seeds[0])
persistence_times = get_persistence_times(**samples, seed=seeds[1])
persistence_times

[251172.00803770928, 105906.80374117418]